In [ ]:
!pip install -U sentence-transformers

     |████████████████████████████████| 71kB 6.8MB/s 
     |████████████████████████████████| 1.5MB 10.1MB/s 
     |████████████████████████████████| 1.1MB 47.6MB/s 
     |████████████████████████████████| 2.9MB 56.6MB/s 
     |████████████████████████████████| 890kB 54.9MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.4.0-cp36-none-any.whl size=102655 sha256=cc008c8f882dcf0cb906c7de40833ef845fb95401892bd027dbcce2c5d310b20
  Stored in directory: /root/.cache/pip/wheels/ff/76/65/50258d8b7930e909ea2f5bd006a23d520a16765af13ab45bb3
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=bcc9fac04155cd2a8ee75c0357f0f035a73f0d5353d71c3331a71e26f5547d91
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [ ]:
import re
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
import math
#from sentence_transformers import LoggingHandler, util
#from sentence_transformers.cross_encoder import CrossEncoder
#from sentence_transformers.cross_encoder.evaluation import CEBinaryClassificationEvaluator
#from sentence_transformers.readers import InputExample
import logging
from datetime import datetime
import os
import gzip
import csv
from zipfile import ZipFile

def preprocess(text):
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text)
    text = re.sub(r'http?:\/\/.*[\r\n]*', '', text)
    text = re.sub(r'[\x03,\xad]', '', text)
    #text = re.sub('[0-9]+', '', text)
    #text = re.sub(r'\s*[一-龥]+', '', text)
    text = re.sub(r'[-=+,___·#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》;}{∼＃°？▶▲▼◀★♥■]', '',text)
    text=" ".join(text.split())
    return text

df = pd.read_csv("/content/drive/My Drive/dacon/news_train.csv") # train.csv 불러오기
test = pd.read_csv("/content/drive/My Drive/dacon/news_test.csv") # test.csv 불러오기
df['clean_text'] = df['content'].apply(preprocess)
test['clean_text'] = test['content'].apply(preprocess)
df['length'] = df['clean_text'].apply(lambda x: len(x))
test['length'] = test['clean_text'].apply(lambda x: len(x))
df['clean_title'] = df['title'].apply(preprocess)
test['clean_title'] = test['title'].apply(preprocess)
train=df.sample(frac=0.85,random_state=200) #0.9
val=df.drop(train.index).sample(frac=1.0)
#### Just some code to print debug information to stdout
#logging.basicConfig(format='%(asctime)s - %(message)s',datefmt='%Y-%m-%d %H:%M:%S',level=logging.INFO, handlers=[LoggingHandler()])

#### data aug   *info(bool)= 0 or 1
train_samples, val_samples, test_samples = [], [], []
train_text = train['clean_text'].values
train_y =  train['info'].values
val_text = val['clean_text'].values
val_y = val['info'].values
test_text = test['clean_text'].values

for row in range(len(train_text)):
    sample = InputExample(texts=[str(train_text[row])], 
                          label=int(train_y[row]))  
    train_samples.append(sample)

for row in range(len(val_text)):
    sample = InputExample(texts=[str(val_text[row])], 
                          label=int(val_y[row]))  
    val_samples.append(sample)

test_text = test['clean_text'].values
test1 = [[test_text[i]] for i in range(0, len(test_text))]
test_title = test['clean_title'].values
test2 = [[test_title[i],test_text[i]] for i in range(0, len(test_text))]

In [ ]:
#Configuration
train_batch_size = 8 #16,32
num_epochs = 3
model = 'monologg/koelectra-base-v3-discriminator'  
#ce-roberta-large-stsb
#kykim/electra-kor-base
model = CrossEncoder()
model = CrossEncoder(model, num_labels=1,device='gpu')
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)
evaluator = CEBinaryClassificationEvaluator.from_input_examples(val_samples)

# Configure the training
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))

model_save_path='content/drive/MyDrive/dacon'
# Train the model
model.fit(train_dataloader=train_dataloader,
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=5000,
          warmup_steps=warmup_steps,
          output_path=model_save_path)

2020-12-29 04:18:41 - Lock 140510689606120 acquired on /root/.cache/huggingface/transformers/353ae9c3d9daa722551a585351b95934cefcf83155f6ff52a1975fa27863dfe0.9c57bd1e7b894b078a3a8ed91a498ca5fb48334c137fe2ec43e8079db1878f8c.lock



2020-12-29 04:18:41 - Lock 140510689606120 released on /root/.cache/huggingface/transformers/353ae9c3d9daa722551a585351b95934cefcf83155f6ff52a1975fa27863dfe0.9c57bd1e7b894b078a3a8ed91a498ca5fb48334c137fe2ec43e8079db1878f8c.lock
2020-12-29 04:18:41 - Lock 140511970239544 acquired on /root/.cache/huggingface/transformers/efc891feb2c6ce810e4350a7fb9d7f9f48d16b78cc24f95c34bd2d5d1d4bb9c4.03eea45db3698b9b1139a0ea93942bdc069b09fd0bb6fd8002f56146135a2cc0.lock



2020-12-29 04:18:48 - Lock 140511970239544 released on /root/.cache/huggingface/transformers/efc891feb2c6ce810e4350a7fb9d7f9f48d16b78cc24f95c34bd2d5d1d4bb9c4.03eea45db3698b9b1139a0ea93942bdc069b09fd0bb6fd8002f56146135a2cc0.lock


Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

2020-12-29 04:18:53 - Lock 140508944840912 acquired on /root/.cache/huggingface/transformers/896eae0fbbb6c2981e5d4559f5d7fc0deb03e81c292abbe5c89b5bb1cf1294b3.541023ff50f833a9bab3e48e78ae1856cf6744bdb336c86e797eaf675b62b2b8.lock



2020-12-29 04:18:53 - Lock 140508944840912 released on /root/.cache/huggingface/transformers/896eae0fbbb6c2981e5d4559f5d7fc0deb03e81c292abbe5c89b5bb1cf1294b3.541023ff50f833a9bab3e48e78ae1856cf6744bdb336c86e797eaf675b62b2b8.lock
2020-12-29 04:18:54 - Lock 140508944841528 acquired on /root/.cache/huggingface/transformers/f2c12b0a8c12c754523b6ed45c1681b4495050720ade30bfe2345e5573d2e141.35f013c4fd3572cfdddbbdf6223ef162dd4fb536bf83007533f201addf3287b7.lock



2020-12-29 04:18:55 - Lock 140508944841528 released on /root/.cache/huggingface/transformers/f2c12b0a8c12c754523b6ed45c1681b4495050720ade30bfe2345e5573d2e141.35f013c4fd3572cfdddbbdf6223ef162dd4fb536bf83007533f201addf3287b7.lock
2020-12-29 04:18:55 - Use pytorch device: cuda
2020-12-29 04:18:55 - Warmup-steps: 3786


2020-12-29 04:28:57 - CEBinaryClassificationEvaluator: Evaluating the model on  dataset in epoch 0 after 5000 steps:
2020-12-29 04:30:20 - Accuracy:           97.96	(Threshold: 0.3137)
2020-12-29 04:30:20 - F1:                 97.45	(Threshold: 0.3137)
2020-12-29 04:30:20 - Precision:          96.06
2020-12-29 04:30:20 - Recall:             98.89
2020-12-29 04:30:20 - Average Precision:  99.73

2020-12-29 04:30:20 - Save model to content/drive/MyDrive/dacon
2020-12-29 04:40:16 - CEBinaryClassificationEvaluator: Evaluating the model on  dataset in epoch 0 after 10000 steps:
2020-12-29 04:41:37 - Accuracy:           98.81	(Threshold: 0.9367)
2020-12-29 04:41:37 - F1:                 98.49	(Threshold: 0.9367)
2020-12-29 04:41:37 - Precision:          98.42
2020-12-29 04:41:37 - Recall:             98.56
2020-12-29 04:41:37 - Average Precision:  99.86

2020-12-29 04:41:37 - Save model to content/drive/MyDrive/dacon

2020-12-29 04:46:45 - CEBinaryClassificationEvaluator: Evaluating the mode

2020-12-29 04:57:56 - CEBinaryClassificationEvaluator: Evaluating the model on  dataset in epoch 1 after 5000 steps:
2020-12-29 04:59:17 - Accuracy:           99.13	(Threshold: 0.0244)
2020-12-29 04:59:17 - F1:                 98.90	(Threshold: 0.0244)
2020-12-29 04:59:17 - Precision:          98.75
2020-12-29 04:59:17 - Recall:             99.05
2020-12-29 04:59:17 - Average Precision:  99.91

2020-12-29 04:59:17 - Save model to content/drive/MyDrive/dacon
2020-12-29 05:09:04 - CEBinaryClassificationEvaluator: Evaluating the model on  dataset in epoch 1 after 10000 steps:
2020-12-29 05:10:24 - Accuracy:           99.21	(Threshold: 0.4133)
2020-12-29 05:10:24 - F1:                 99.00	(Threshold: 0.4133)
2020-12-29 05:10:24 - Precision:          98.78
2020-12-29 05:10:24 - Recall:             99.23
2020-12-29 05:10:24 - Average Precision:  99.92

2020-12-29 05:10:24 - Save model to content/drive/MyDrive/dacon

2020-12-29 05:15:35 - CEBinaryClassificationEvaluator: Evaluating the mode

2020-12-29 05:26:45 - CEBinaryClassificationEvaluator: Evaluating the model on  dataset in epoch 2 after 5000 steps:
2020-12-29 05:28:06 - Accuracy:           99.23	(Threshold: 0.0578)
2020-12-29 05:28:06 - F1:                 99.02	(Threshold: 0.0213)
2020-12-29 05:28:06 - Precision:          98.98
2020-12-29 05:28:06 - Recall:             99.06
2020-12-29 05:28:06 - Average Precision:  99.92

2020-12-29 05:37:52 - CEBinaryClassificationEvaluator: Evaluating the model on  dataset in epoch 2 after 10000 steps:
2020-12-29 05:39:13 - Accuracy:           99.21	(Threshold: 0.1152)
2020-12-29 05:39:13 - F1:                 99.00	(Threshold: 0.0202)
2020-12-29 05:39:13 - Precision:          98.84
2020-12-29 05:39:13 - Recall:             99.17
2020-12-29 05:39:13 - Average Precision:  99.91


2020-12-29 05:44:18 - CEBinaryClassificationEvaluator: Evaluating the model on  dataset after epoch 2:
2020-12-29 05:45:38 - Accuracy:           99.20	(Threshold: 0.1261)
2020-12-29 05:45:38 - F1:      

In [ ]:
model.save_pretrained('cross_encoder1')  

2020-12-29 07:38:24 - Save model to cross_encoder1


In [ ]:
scores1 = model.predict(test1,batch_size=16)

In [ ]:
scores2 = model.predict(test2,batch_size=16)

In [ ]:
df_describe = pd.DataFrame(data = [scores1,scores2]).T
df_describe.describe()

,0,1
count,142565.000000,142565.000000
mean,0.382605,0.392593
std,0.485038,0.485687
min,0.000005,0.000005
25%,0.000006,0.000009
50%,0.000007,0.000051
75%,0.999995,0.999988
max,0.999996,0.999996


In [ ]:
count = 0
l1 = []
for i in range(len(scores)):
  if abs(scores[i]-scores1[i])>0.9:
    count += 1
    l1.append(i)
print(count)

6076


In [ ]:
sample_sub=pd.read_csv('/content/drive/My Drive/dacon/sample_submission.csv')
x2 = [1 if scores1[i]>0.5 else 0 for i in range(len(scores1))]
sample_sub['info']=x2
sample_sub.to_csv('/content/drive/My Drive/dacon/encoder_train_text1.csv',index=False)

In [ ]:
sample_sub

,id,info
0,NEWS00237_1,1
1,NEWS00237_2,0
2,NEWS00237_3,0
3,NEWS00237_4,0
4,NEWS00237_5,0
...,...,...
142560,NEWS09482_72,1
142561,NEWS09482_73,1
142562,NEWS09482_74,1
142563,NEWS09482_75,1
